<a href="https://colab.research.google.com/github/aksangha2/image_captioning/blob/master/image_captioning_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aksangha","key":"8d07e2f779d8485b93c3de1ffff85f2c"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d hsankesara/flickr-image-dataset -p/content

100% 8.15G/8.16G [03:09<00:00, 18.5MB/s]
100% 8.16G/8.16G [03:09<00:00, 46.2MB/s]


In [4]:
! unzip /content/flickr-image-dataset.zip -d train_data

Streaming output truncated to the last 5000 lines.
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/535529555.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/5356236664.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/5356318668.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/535830521.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/5359437945.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/5361355399.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/53614287.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/536147209.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/5361551308.jpg  
  inflating: train_data/flickr30k_images/flickr30k_images/flickr30k_images/536156200.jpg  
  inflating: train_data/flickr30k_i

In [0]:
import pandas as pd
import re
import json
import collections
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

In [0]:
from keras.applications.resnet import ResNet50,preprocess_input
from keras.layers import *
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


In [14]:
captions_df=pd.read_csv("train_data/flickr30k_images/results.csv",header=None,error_bad_lines=False)

b'Skipping line 3: expected 1 fields, saw 2\nSkipping line 22: expected 1 fields, saw 4\nSkipping line 39: expected 1 fields, saw 2\nSkipping line 48: expected 1 fields, saw 2\nSkipping line 68: expected 1 fields, saw 2\nSkipping line 92: expected 1 fields, saw 4\nSkipping line 93: expected 1 fields, saw 3\nSkipping line 94: expected 1 fields, saw 2\nSkipping line 97: expected 1 fields, saw 3\nSkipping line 99: expected 1 fields, saw 2\nSkipping line 101: expected 1 fields, saw 2\nSkipping line 117: expected 1 fields, saw 2\nSkipping line 133: expected 1 fields, saw 2\nSkipping line 187: expected 1 fields, saw 3\nSkipping line 202: expected 1 fields, saw 3\nSkipping line 208: expected 1 fields, saw 2\nSkipping line 222: expected 1 fields, saw 2\nSkipping line 238: expected 1 fields, saw 2\nSkipping line 241: expected 1 fields, saw 2\nSkipping line 247: expected 1 fields, saw 3\nSkipping line 252: expected 1 fields, saw 2\nSkipping line 278: expected 1 fields, saw 3\nSkipping line 292: 

In [0]:
captions=(captions_df.values)[1:]

In [0]:
descriptions={}
for x in captions:
  x=x[0].split("|")
  img_name=x[0]
  if descriptions.get(img_name) is None:
    descriptions[img_name]=[]
  if(2<len(x)):
    descriptions[img_name].append(x[2])

In [0]:
def clean_text(sentence):
  sentence=sentence.lower()
  sentence=re.sub("[^a-z]+"," ",sentence)
  sentence=sentence.split()
  sentence=[s for s in sentence if len(s)>1]
  sentence=" ".join(sentence)
  return(sentence)

In [0]:
for key,captions in descriptions.items():
  for i in range(len(captions)):
    captions[i]=clean_text(captions[i])

In [0]:
with open('descriptions.txt','w') as f:
  f.write(str(descriptions))

In [0]:
vocab=set()
for key in descriptions.keys():
  [vocab.update(sentence.split()) for sentence in descriptions[key]]

total_words=[]
for key in descriptions.keys():
  [total_words.append(i) for des in descriptions[key] for i in des.split()]

counter=collections.Counter(total_words)
freq_cnt=dict(counter)
sorted_freq_cnt=sorted(freq_cnt.items(),reverse=True,key=lambda x:x[1])

threshold=5
sorted_freq_cnt=[x for x in sorted_freq_cnt if x[1]>threshold]
total_words=[x[0] for x in sorted_freq_cnt]


PREPARE TRAIN TEST DATA

In [0]:
train_descriptions={}
for img_id in descriptions.keys():
  train_descriptions[img_id]=[]
  for cap in descriptions[img_id]:
    cap_to_append="startseq "+cap+" endseq"
    train_descriptions[img_id].append(cap_to_append)

In [0]:
#preprocessing captions
#convert captions to numeric values

In [0]:
idx_to_word={}
word_to_idx={}
for i,word in enumerate(total_words):
  word_to_idx[word]=i+1
  idx_to_word[i+1]=word
last_idx=len(idx_to_word)
idx_to_word[last_idx+1]='startseq '
word_to_idx['startseq ']=last_idx+1
idx_to_word[last_idx+2]=' endseq'
word_to_idx[' endseq']=last_idx+2
vocab_size=len(word_to_idx)+1
max_len=0
for key in train_descriptions.keys():
  for cap in train_descriptions[key]:
    max_len=max(max_len,len(cap.split()))

IMAGE PREPROCESSING

In [27]:
model=ResNet50(weights='imagenet',input_shape=(224,224,3))
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
model_new=Model(model.input,model.layers[-2].output)

In [0]:
def preprocess_image(img):
  img=image.load_img(img,target_size=(224,224))
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  img=preprocess_input(img)
  return img

In [0]:
img_path="train_data/flickr30k_images/flickr30k_images/"
img=preprocess_image(img_path+"5380411758.jpg")
def encode_image(img):
  img=preprocess_image(img)
  feature_vector=model_new.predict(img)
  feature_vector=feature_vector.reshape((-1,))
  return feature_vector

In [32]:
encoding_train={}
start=time.time()
ix=1
for key in train_descriptions.keys():
  path=img_path+key
  encoding_train[key]=encode_image(path)
  ix+=1
  if ix%1000==0:
    print("encoding in progress time step%d"%ix)
end_t=time.time()

encoding in progress time step1000
encoding in progress time step2000
encoding in progress time step3000
encoding in progress time step4000
encoding in progress time step5000
encoding in progress time step6000
encoding in progress time step7000
encoding in progress time step8000
encoding in progress time step9000
encoding in progress time step10000
encoding in progress time step11000
encoding in progress time step12000
encoding in progress time step13000
encoding in progress time step14000
encoding in progress time step15000
encoding in progress time step16000
encoding in progress time step17000
encoding in progress time step18000
encoding in progress time step19000
encoding in progress time step20000
encoding in progress time step21000
encoding in progress time step22000
encoding in progress time step23000
encoding in progress time step24000
encoding in progress time step25000
encoding in progress time step26000
encoding in progress time step27000
encoding in progress time step28000
e

In [33]:
print("total time taken:",end_t-start)

total time taken: 1286.0590600967407


In [0]:
with open("encoding_train_features.pkl",'wb') as f:
  pickle.dump(encoding_train,f)

In [0]:
#text preprocessing

In [0]:
#descriptions=None
#with open('descriptions.txt','r') as f:
#  descriptions=f.read()
#f.close()

In [0]:
#json_acceptable_string=descriptions.replace("'","\"")
#descriptions=json.loads(json_acceptable_string)

TRAINING

In [0]:
def data_generator(train_descriptions,encoding_train,word_to_idx,max_len,batch_size):
  x1,x2,y=[],[],[]
  n=0
  while True:
    for key,desc_list in train_descriptions.items():
      n+=1
      photo=encoding_train[key]
      for desc in desc_list:
        seq=[word_to_idx[word] for word in desc.split() if word in word_to_idx]
        for i in range(1,len(seq)):
          xi=seq[0:i]
          yi=seq[i]
          xi=pad_sequences([xi],maxlen=max_len,value=0,padding='post')[0]
          yi=to_categorical([yi],num_classes=vocab_size)[0]
          x1.append(photo)
          x2.append(xi)
          y.append(yi)
        if n==batch_size:
          yield[[np.array(x1),np.array(x2)],np.array(y)]
          x1,x2,y=[],[],[]
          n=0

In [36]:
!kaggle datasets download -d watts2/glove6b50dtxt -p/content

 84% 57.0M/67.7M [00:02<00:00, 11.7MB/s]
100% 67.7M/67.7M [00:02<00:00, 30.6MB/s]


In [37]:
! unzip glove6b50dtxt.zip

Archive:  glove6b50dtxt.zip
  inflating: glove.6B.50d.txt        


In [0]:
f=open("glove.6B.50d.txt",encoding='utf8')

In [0]:
embedding_index={}
for line in f:
  values=line.split()
  word=values[0]
  embedding_word=np.array(values[1:],dtype='float')
  embedding_index[word]=embedding_word
f.close()

In [0]:
def get_embedding_matrix():
  emb_dim=50
  matrix=np.zeros((vocab_size,emb_dim))
  for word,idx in word_to_idx.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
      matrix[idx]=embedding_vector
  return matrix

In [0]:
embedding_matrix=get_embedding_matrix()

MODEL ARCHITECTURE

In [0]:
input_img_features=Input(shape=(2048,))
inp_img1=Dropout(0.3)(input_img_features)
inp_img2=Dense(256,activation='relu')(inp_img1)

In [0]:
input_captions=Input(shape=(max_len,))
inp_cap1=Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2=Dropout(0.3)(inp_cap1)
inp_cap3=LSTM(256)(inp_cap2)

In [0]:
decoder1=add([inp_img2,inp_cap3])
decoder2=Dense(256,activation='relu')(decoder1)

In [0]:
outputs=Dense(vocab_size,activation='softmax')(decoder2)

In [59]:
model=Model(inputs=[input_img_features,input_captions],outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 70, 50)       318800      input_6[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_5[0][0]                    
____________________________________________________________________________________________

In [0]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable=False

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

TRAINING

In [0]:
epochs=20
batch_size=3
steps=len(train_descriptions)

In [0]:
for i in range(epochs):
  generator=data_generator(train_descriptions,encoding_train,word_to_idx,max_len,batch_size)
  model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
  model.save(model_weights/model_+str(i)+'.h5')

Epoch 1/1
 2714/31780 [=>............................] - ETA: 1:24:41 - loss: 3.8846

PREDICTIONS

In [0]:
def predict_caption(photo):
  in_text='startseq '
  for i in range(max_len):
    sequence=[word_to_idx[w] for w in in_text.split() if w in word_to_idx]
    sequence=pad_sequence([sequence],maxlen=max_len,padding='post')
    ypred=model.predict([photo,sequence])
    ypred=ypred.argmax()
    word=idx_to_word[ypred]
    in_text+=' '+word
    if word==' endseq':
      break 
  final_caption=in_text.split()[1:-1]
  final_caption=' '.join(final_caption)
  return final_caption

In [0]:
img_test_path=" "
photo=encode_image(img_test_path)
photo=np.reshape(photo,(1,2048))
pred_caption=predict_caption(photo)

In [64]:
!pip install catboost

     |████████████████████████████████| 64.4MB 1.2MB/s 
